In [2]:
import torch
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F


In [3]:
import dlc_practical_prologue as prologue


In [4]:
train_input, train_target, test_input, test_target = \
    prologue.load_data(one_hot_labels = True, normalize = True, flatten = False)


* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x


In [6]:
train_input, train_target = Variable(train_input), Variable(train_target)


In [7]:
def train_model(model, train_input, train_target, mini_batch_size, eta=1e-1, 
                num_epochs=25, criterion=nn.MSELoss()):
    for e in range(0, num_epochs):
        sum_loss = 0
        # We do this with mini-batches
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            sum_loss = sum_loss + loss.data[0]
            model.zero_grad()
            loss.backward()
            for p in model.parameters():
                p.data.sub_(eta * p.grad.data)
        print(e, sum_loss)


In [8]:
model = Net()

In [9]:
train_model(model, train_input, train_target, 100)

0 4.938808262348175
1 3.8027787804603577
2 3.3847336173057556
3 2.9938721656799316
4 2.632626920938492
5 2.696588456630707
6 2.39801986515522
7 2.2006585597991943
8 1.9581044614315033
9 1.8174351751804352
10 1.6805791854858398
11 1.4897380024194717
12 1.4427620023488998
13 1.3860284313559532
14 1.329150602221489
15 1.1258902177214622
16 1.1413678601384163
17 1.048056036233902
18 1.0363075882196426
19 0.9429718628525734
20 0.954571396112442
21 0.8853479400277138
22 0.7955501079559326
23 0.7548313327133656
24 0.8152499198913574


In [10]:
test_target.shape

torch.Size([1000, 10])

In [11]:
test_input.shape

torch.Size([1000, 1, 28, 28])

In [12]:
test_input, test_target = Variable(test_input), Variable(test_target)


In [13]:
test_input[0].size()

torch.Size([1, 28, 28])

In [14]:
tmp_pred = model(test_input.narrow(0, 1, 1))

In [15]:
tmp_pred

Variable containing:
-0.4376 -0.6748 -0.0570 -0.9269 -1.1562 -1.2176 -0.5933 -1.2256 -0.6457 -1.2463
[torch.cuda.FloatTensor of size 1x10 (GPU 0)]

In [16]:
val= tmp_pred.max()

In [17]:
max_idx = tmp_pred==val

In [18]:
max_idx

Variable containing:
    0     0     1     0     0     0     0     0     0     0
[torch.cuda.ByteTensor of size 1x10 (GPU 0)]

In [19]:
max_idx = max_idx.float()

In [21]:
tmp_ans = test_target.narrow(0,1, 1)

In [22]:
tmp_batch_ans = test_target.narrow(0,1, 30)

In [23]:
tmp_batch_ans

Variable containing:
   -1    -1     1    -1    -1    -1    -1    -1    -1    -1
   -1     1    -1    -1    -1    -1    -1    -1    -1    -1
    1    -1    -1    -1    -1    -1    -1    -1    -1    -1
   -1    -1    -1    -1     1    -1    -1    -1    -1    -1
   -1     1    -1    -1    -1    -1    -1    -1    -1    -1
   -1    -1    -1    -1     1    -1    -1    -1    -1    -1
   -1    -1    -1    -1    -1    -1    -1    -1    -1     1
   -1    -1    -1    -1    -1     1    -1    -1    -1    -1
   -1    -1    -1    -1    -1    -1    -1    -1    -1     1
    1    -1    -1    -1    -1    -1    -1    -1    -1    -1
   -1    -1    -1    -1    -1    -1     1    -1    -1    -1
   -1    -1    -1    -1    -1    -1    -1    -1    -1     1
    1    -1    -1    -1    -1    -1    -1    -1    -1    -1
   -1     1    -1    -1    -1    -1    -1    -1    -1    -1
   -1    -1    -1    -1    -1     1    -1    -1    -1    -1
   -1    -1    -1    -1    -1    -1    -1    -1    -1     1
   -1    -1    -1  

In [24]:
tmp_batch_pred = model(test_input.narrow(0, 1, 30))

In [25]:
tmp_batch_pred

Variable containing:
-0.4376 -0.6748 -0.0570 -0.9269 -1.1562 -1.2176 -0.5933 -1.2256 -0.6457 -1.2463
-0.8897  0.7963 -1.0881 -0.8873 -0.8911 -0.9727 -0.9410 -0.7041 -0.9054 -1.0263
 0.8021 -1.1225 -1.2650 -0.9816 -1.0224 -1.3251 -0.7535 -0.7151 -1.0444 -0.8582
-1.0378 -0.8712 -1.1804 -0.9227  0.7460 -1.2810 -0.8172 -0.5629 -0.9938 -0.9377
-0.9154  0.8726 -1.2129 -0.9611 -1.0083 -1.2355 -1.0351 -0.6877 -0.9813 -0.9960
-1.2860 -0.8813 -1.2562 -0.8147  0.3782 -1.4103 -0.8804 -0.5972 -0.6416 -0.9620
-0.9324 -0.9687 -0.8900 -0.8830 -0.6821 -1.0232 -0.7809 -0.6995 -0.5379 -0.1058
-0.9662 -1.0657 -0.3847 -1.3716 -0.7667 -0.2075 -0.5186 -0.8074 -0.7190 -0.7214
-0.9722 -1.1895 -1.1869 -0.8815 -0.7913 -1.3692 -0.9676 -0.3487 -0.4993  0.2223
 0.7219 -1.0087 -0.9736 -1.0727 -0.8711 -1.0938 -0.9293 -0.9840 -0.6630 -1.0484
-0.4819 -1.2164 -1.2290 -0.8915 -0.6528 -1.1434  0.2278 -1.0230 -0.2944 -0.9053
-0.7830 -1.1018 -1.0193 -1.0171 -0.6045 -1.2747 -1.0809 -0.7769 -0.9118  0.6018
 1.0382 -0.9740 -1.

In [26]:
tmp_batch_pred.max(1)[1]

Variable containing:
 2
 1
 0
 4
 1
 4
 9
 5
 9
 0
 6
 9
 0
 1
 5
 9
 7
 3
 4
 9
 6
 6
 5
 4
 0
 7
 4
 0
 1
 3
[torch.cuda.LongTensor of size 30 (GPU 0)]

In [27]:
tmp_batch_ans.max(1)

(Variable containing:
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
 [torch.cuda.FloatTensor of size 30 (GPU 0)], Variable containing:
  2
  1
  0
  4
  1
  4
  9
  5
  9
  0
  6
  9
  0
  1
  5
  9
  7
  3
  4
  9
  6
  6
  5
  4
  0
  7
  4
  0
  1
  3
 [torch.cuda.LongTensor of size 30 (GPU 0)])

In [28]:
tmp_batch_ans.max(1)[1]

Variable containing:
 2
 1
 0
 4
 1
 4
 9
 5
 9
 0
 6
 9
 0
 1
 5
 9
 7
 3
 4
 9
 6
 6
 5
 4
 0
 7
 4
 0
 1
 3
[torch.cuda.LongTensor of size 30 (GPU 0)]

In [29]:
tmp_batch_pred.max(1)[1]

Variable containing:
 2
 1
 0
 4
 1
 4
 9
 5
 9
 0
 6
 9
 0
 1
 5
 9
 7
 3
 4
 9
 6
 6
 5
 4
 0
 7
 4
 0
 1
 3
[torch.cuda.LongTensor of size 30 (GPU 0)]

In [30]:
res = tmp_batch_ans.max(1)[1] != tmp_batch_pred.max(1)[1]

In [31]:
res.sum().data


 0
[torch.cuda.ByteTensor of size 1 (GPU 0)]

In [33]:
tmp_ans

Variable containing:
   -1    -1     1    -1    -1    -1    -1    -1    -1    -1
[torch.cuda.FloatTensor of size 1x10 (GPU 0)]

In [37]:
def compute_nb_errors(model, test_input, test_target, mini_batch_size):
    nb_error = 0.0
    for b in range(0, test_input.size(0), mini_batch_size):
        test_output = model(test_input.narrow(0, b, mini_batch_size))
        test_ans = test_target.narrow(0, b, mini_batch_size)
        output_max_idx = test_output.max(1)[1]
        ans_max_idx = test_ans.max(1)[1]
        nb_error +=  (output_max_idx!=ans_max_idx).sum()
    return nb_error

In [39]:
compute_nb_errors(model, test_input, test_target, 100)

Variable containing:
 96
[torch.cuda.ByteTensor of size 1 (GPU 0)]